In [1]:
from transformers import pipeline

print("--- TAREA EXTRA: ZERO-SHOT CLASSIFICATION ---")
print("Objetivo: Detectar el TIPO de toxicidad sin re-entrenar el modelo.\n")

#Cargamos el pipeline (Baja el modelo automáticamente)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

#Etiquetas que queremos probar 
candidate_labels = ["insult", "threat", "hate speech", "constructive criticism"]

#Ejemplos de tu dataset o inventados
ejemplos = [
     "i will let you die",
    "fire black athletes",
    "stupid people are everywhere",
    "you are a liar and stupid",
    "many times want to validate stupid actions",
    "trying to get shit done"
]

for text in ejemplos:
    result = classifier(text, candidate_labels)

    print(f"Texto: '{text}'")
    print(f"Ganador: {result['labels'][0]} (Confianza: {result['scores'][0]:.2%})")
    print("-" * 30)


c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- TAREA EXTRA: ZERO-SHOT CLASSIFICATION ---
Objetivo: Detectar el TIPO de toxicidad sin re-entrenar el modelo.



c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\js834\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


Texto: 'i will let you die'
Ganador: threat (Confianza: 85.69%)
------------------------------
Texto: 'fire black athletes'
Ganador: threat (Confianza: 79.39%)
------------------------------
Texto: 'stupid people are everywhere'
Ganador: threat (Confianza: 52.71%)
------------------------------
Texto: 'you are a liar and stupid'
Ganador: insult (Confianza: 75.56%)
------------------------------
Texto: 'many times want to validate stupid actions'
Ganador: threat (Confianza: 48.79%)
------------------------------
Texto: 'trying to get shit done'
Ganador: threat (Confianza: 75.93%)
------------------------------


In [2]:
from transformers import pipeline

# Cargar pipeline de emociones
classifier_emotion = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

ejemplos = ["I hope you die soon", "This game is so frustrating but I love it", "Eww, get away from me"]

# Probar
for texto in ejemplos:
    resultado = classifier_emotion(texto)
    # Mostrar la emoción ganadora
    print(f"Texto: '{texto}' -> Emoción: {resultado[0][0]['label']} ({resultado[0][0]['score']:.2f})")

c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\js834\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is no

Texto: 'I hope you die soon' -> Emoción: neutral (0.64)
Texto: 'This game is so frustrating but I love it' -> Emoción: sadness (0.48)
Texto: 'Eww, get away from me' -> Emoción: disgust (0.55)


In [3]:
from transformers import pipeline

print("--- ANÁLISIS DE OBJETIVOS (NER) ---")
print("Objetivo: Identificar a QUÉ o QUIÉN se ataca (Personas, Organizaciones, Países).\n")

# 1. Cargar el pipeline de NER (Named Entity Recognition)
# aggregation_strategy="simple" es CLAVE: junta los tokens ("J", "##ohn") en palabras completas ("John")
ner_classifier = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

# 2. Ejemplos con entidades claras
ejemplos_ner = [
    "I hate John because he is stupid",              # Ataque a Persona
    "Google is the worst company ever",              # Ataque a Organización
    "I will destroy Paris and all its people",       # Ataque a Lugar
    "Americans are always so loud",                  # Ataque a Grupo (Gentilicio)
    "Kill Elon Musk right now"                       # Amenaza directa a Persona
]

# 3. Procesar y mostrar resultados
for texto in ejemplos_ner:
    entidades = ner_classifier(texto)
    
    print(f"Frase: '{texto}'")
    if len(entidades) > 0:
        for ent in entidades:
            # ent['entity_group'] te dice si es PER (Persona), ORG (Organización), LOC (Lugar), etc.
            print(f"  └── 🎯 Entidad detectada: {ent['word']} (Tipo: {ent['entity_group']}, Confianza: {ent['score']:.2f})")
    else:
        print("  └── No se detectaron nombres propios específicos.")
    print("-" * 30)

--- ANÁLISIS DE OBJETIVOS (NER) ---
Objetivo: Identificar a QUÉ o QUIÉN se ataca (Personas, Organizaciones, Países).



c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\js834\.cache\huggingface\hub\models--dslim--bert-base-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back 

Frase: 'I hate John because he is stupid'
  └── 🎯 Entidad detectada: John (Tipo: PER, Confianza: 1.00)
------------------------------
Frase: 'Google is the worst company ever'
  └── 🎯 Entidad detectada: Google (Tipo: ORG, Confianza: 1.00)
------------------------------
Frase: 'I will destroy Paris and all its people'
  └── 🎯 Entidad detectada: Paris (Tipo: LOC, Confianza: 1.00)
------------------------------
Frase: 'Americans are always so loud'
  └── 🎯 Entidad detectada: Americans (Tipo: MISC, Confianza: 1.00)
------------------------------
Frase: 'Kill Elon Musk right now'
  └── 🎯 Entidad detectada: El (Tipo: PER, Confianza: 0.92)
  └── 🎯 Entidad detectada: ##on Musk (Tipo: PER, Confianza: 0.86)
------------------------------


In [1]:
# Supongamos que tienes los resultados del Zero-Shot Classification del notebook 4c
# Simulamos datos de tipos de toxicidad detectados
tipos_toxicidad = ['Insulto', 'Amenaza', 'Lenguaje de Odio', 'Sarcasmo']
conteos = [450, 120, 300, 130] # Ejemplo

plt.figure(figsize=(10, 6))
barplot = sns.barplot(x=tipos_toxicidad, y=conteos, palette='viridis')
plt.title('Análisis de Tipos de Toxicidad Detectados (Zero-Shot)', fontsize=15)
plt.ylabel('Frecuencia')
plt.xlabel('Categoría')

# Añadir etiquetas
for p in barplot.patches:
    barplot.annotate(format(p.get_height(), '.0f'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')
plt.show()

NameError: name 'plt' is not defined